### import drive from google.colab 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **Change directry to where training data is save....**

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
!ls

1   15	20  26	31  8				 model_400_400_170_resnet50.h5
10  16	21  27	32  9
11  17	22  28	4   model_256_256_resnet.h5
12  18	23  29	5   model_300_400_11_vgg19_2.h5
13  19	24  3	6   model_300_400_11_vgg19.h5
14  2	25  30	7   model_400_400_10_vgg19.h5


*Import libraries like-italicized text
numpy for make matrix..
pandas for reading csv file
csv for make a csv file
matplotlib.pyplot for ploting and visualization
ImageDataGenerator for making traing and test data
os to change directry*

In [0]:
import numpy as np
import pandas as pd
import csv
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


made a datagenerator

In [0]:
datagen = ImageDataGenerator(rescale=1./255,
                            shear_range=0.4,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            validation_split=0.05)

split the training data in train and validation 
and the parameter batch size = 32
and resize the image to 256*256*3


In [0]:
batch_size = 32
img_shape = (300,400,3)
img_hight = 400
img_width = 300
train_data = datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/coin_train_x',
                                    target_size = (img_width,img_hight),
                                    class_mode = 'categorical',
                                    batch_size=batch_size,
                                    shuffle=True,
                                   subset='training')

validation_data = datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/coin_train_x',
                                             target_size=(img_width,img_hight),
                                             class_mode='categorical',
                                             batch_size=batch_size,
                                             shuffle=True,
                                              subset='validation')

Found 6695 images belonging to 32 classes.
Found 335 images belonging to 32 classes.


In [0]:
import tensorflow as tf
keras = tf.keras

*Load the pretrained VGG19 model without top layer*

In [0]:
base_model = tf.keras.applications.VGG16(input_shape = img_shape,include_top = False,weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
58892288/58889256 [==============================] - 5s 0us/step


*Set the layer of model to be trained*

In [0]:
trainable_layers = 5
for layer in base_model.layers[:trainable_layers]:
  layer.trainable=False
for layer in base_model.layers[trainable_layers:]:
  layer.trainable=True

*Print the summary about the VGG19 model*

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 400, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 200, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 100, 128)      0     

*define the a layer global average layer which converts (8,8,512) to (1,1,512)*

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(feature_batch)
#print(feature_batch_average.shape)

*define the prediction_layer with class equal to 32*

In [0]:
middle = keras.layers.Dense(64,activation='relu')
prediction_layer = keras.layers.Dense(32,activation='softmax')
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

*Made the final Model*

In [0]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])


*Set the value of learning rate and compile the model for training*

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

*to save the trian model weights at every step*

In [0]:
checkpoint_path = "/cp_more_layer_11.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

/


*Train the model with steps_per_epoch = 625,
epoch = 2*

In [0]:
model.fit_generator(train_data,
                   steps_per_epoch = 625,
                   validation_data = validation_data,
                   validation_steps = 100,
                   callbacks=[cp_callback], 
                   epochs=2)
os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
model.save_weights("model_300_400_11_vgg19.h5")

Epoch 1/2
624/625 [============================>.] - ETA: 5s - loss: 1.4739 - acc: 0.6071 
Epoch 00001: saving model to /cp_more_layer_11.ckpt
625/625 [==============================] - 3850s 6s/step - loss: 1.4717 - acc: 0.6078 - val_loss: 0.4546 - val_acc: 0.8700
Epoch 2/2
624/625 [============================>.] - ETA: 1s - loss: 0.2451 - acc: 0.9336
Epoch 00002: saving model to /cp_more_layer_11.ckpt
625/625 [==============================] - 1071s 2s/step - loss: 0.2450 - acc: 0.9336 - val_loss: 0.2787 - val_acc: 0.9304


In [0]:
model.fit_generator(train_data,
                   steps_per_epoch = 512,
                   validation_data = validation_data,
                   validation_steps = 100,
                   callbacks=[cp_callback], 
                   epochs=2)
os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
model.save_weights("model_300_400_11_vgg19.h5")

Epoch 1/2
511/512 [============================>.] - ETA: 1s - loss: 1.6492 - acc: 0.5540
Epoch 00001: saving model to /cp_more_layer_11.ckpt
512/512 [==============================] - 580s 1s/step - loss: 1.6470 - acc: 0.5545 - val_loss: 0.9859 - val_acc: 0.7562
Epoch 2/2
511/512 [============================>.] - ETA: 1s - loss: 0.4745 - acc: 0.8744
Epoch 00002: saving model to /cp_more_layer_11.ckpt
512/512 [==============================] - 577s 1s/step - loss: 0.4746 - acc: 0.8742 - val_loss: 0.5705 - val_acc: 0.8691


In [0]:
model.fit_generator(train_data,
                   steps_per_epoch = 512,
                   validation_data = validation_data,
                   validation_steps = 100,
                   callbacks=[cp_callback], 
                   epochs=1)
#os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
#model.save_weights("model_300_400_11_vgg19_2.h5")

511/512 [============================>.] - ETA: 1s - loss: 0.0345 - acc: 0.9905
Epoch 00001: saving model to /cp_more_layer_11.ckpt
512/512 [==============================] - 1066s 2s/step - loss: 0.0345 - acc: 0.9905 - val_loss: 0.3332 - val_acc: 0.9442


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
model.save_weights("model_300_400_11_vgg19_2.h5")

In [0]:
datage = ImageDataGenerator(rescale=1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

In [0]:
test_data = datage.flow_from_directory('/content/drive/My Drive/Colab Notebooks/coin_test_x',
                                    target_size = (img_width,img_hight),
                                    class_mode = 'categorical',
                                    batch_size=batch_size,
                                    shuffle=True,
                                   subset='training')

Found 840 images belonging to 32 classes.


In [0]:
model.evaluate_generator(generator=test_data,
steps=200)

[0.28370814798778154, 0.9444907]

In [0]:
!ls {checkpoint_dir}

bin	 datalab  home	 lib64	opt   run   swift		  tmp	 var
boot	 dev	  lib	 media	proc  sbin  sys			  tools
content  etc	  lib32  mnt	root  srv   tensorflow-2.0.0-rc0  usr


In [0]:
loaded_model =  tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [0]:
base_learning_rate = 0.0001
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
loaded_model.load_weights("model_300_400_11_vgg19_2.h5")

In [0]:
loaded_model.evaluate_generator(generator=test_data,
steps=200)

[0.3219913749702391, 0.9287548]

In [0]:
# loaded_model.fit_generator(train_data,
#                    steps_per_epoch = 512,
#                    validation_data = validation_data,
#                    validation_steps = 100,
#                    callbacks=[cp_callback], 
#                    epochs=1)
# os.chdir("/content/drive/My Drive/Colab Notebooks/coin_train_x")
# model.save_weights("model_300_400_11_vgg19_2.h5")

511/512 [============================>.] - ETA: 2s - loss: 0.1099 - acc: 0.9682
Epoch 00001: saving model to /cp_more_layer_11.ckpt
512/512 [==============================] - 1822s 4s/step - loss: 0.1100 - acc: 0.9682 - val_loss: 0.4984 - val_acc: 0.9101


511/512 [============================>.] - ETA: 1s - loss: 0.0737 - acc: 0.9788
Epoch 00001: saving model to /cp_more_layer_11.ckpt
512/512 [==============================] - 602s 1s/step - loss: 0.0736 - acc: 0.9788 - val_loss: 0.4432 - val_acc: 0.9170


In [0]:
pre =loaded_model.predict(test_data)

In [0]:
print(pre)

[[7.0775153e-18 6.2315740e-13 1.6489217e-16 ... 1.0000000e+00
  5.4846146e-14 8.6441999e-12]
 [9.9917620e-01 2.3847322e-08 2.7899960e-09 ... 3.5976064e-09
  1.4284453e-07 6.2494165e-09]
 [1.4427103e-15 2.4669288e-13 1.8831858e-11 ... 6.9326218e-13
  5.5046858e-13 1.6916329e-14]
 ...
 [7.9348688e-09 1.7813227e-11 1.6832709e-10 ... 6.2383004e-10
  2.4125796e-08 1.4191423e-05]
 [9.9973840e-01 5.3983928e-08 2.2728829e-10 ... 2.1123743e-10
  4.6332262e-09 1.2576325e-14]
 [1.5576409e-08 2.7113544e-11 4.8964317e-08 ... 1.8729367e-07
  4.1835233e-07 3.8672523e-09]]


In [0]:
os.chdir("/content/drive/My Drive/test")

In [0]:
dic = ['australia', 'brazil', 'united_kingdom', 'canada', 'chile', 'china', 'czech_republic', 'denmark', 'spain', 'hong_kong', 'hungary', 'india', 'indonesia', 'israel', 'japan', 'south_korea', 'malaysia', 'mexico', 'new_zealand', 'norway', 'pakistan', 'philippines', 'poland', 'russia', 'singapore', 'south_africa', 'sweden', 'switzerland', 'taiwan', 'thailand', 'turkey', 'usa']

In [0]:
import glob
import csv
idx2country = {}
for z in range(1,33):
  test_path = '/content/drive/My Drive/Colab Notebooks/coin_train_x/' + str(z)
  di = {}
  for img in os.listdir(test_path):
    idx = img.split('.')[0]
    l = img.split('.')[0]
    p = l.split(' ')[-1]
    q = p.split('_')
    q = q[1:]
    k = ""
    if len(q) == 2:
      k = q[0] + '_' + q[1]
    else:
      k = q[0]
  
    img_path = os.path.join(test_path,img)
    IMG = cv2.imread(img_path)
    IMG = cv2.resize(IMG, (256,256))
    IMG = np.array(IMG).astype('float32') / 255.
    IMG = np.expand_dims(IMG, axis = 0)
    IMG = np.vstack([IMG])
    #pred = loaded_model.predict([IMG])
    #Id = np.where(pred.max)
    Class = model.predict_classes(IMG, batch_size=1)
    ID = str(int(Class[0])+1)
    print(ID)
    print(k)
    idx2country[Class[0]+1] = k
    print("##########################")
    break
    #if ID == '0':
      #print("Error")
    #print (idx2country[ID])
    #di[idx] = idx2country[ID]


1
australia
##########################
12
brazil
##########################
23
united_kingdom
##########################
27
canada
##########################
28
chile
##########################
29
china
##########################
30
czech_republic
##########################
31
denmark
##########################
32
spain
##########################
2
hong_kong
##########################
3
hungary
##########################
4
india
##########################
5
indonesia
##########################
6
israel
##########################
7
japan
##########################
8
south_korea
##########################
9
malaysia
##########################
10
mexico
##########################
11
new_zealand
##########################
13
norway
##########################
14
pakistan
##########################
15
philippines
##########################
16
poland
##########################
17
russia
##########################
18
singapore
##########################
19
south_africa
########################

In [0]:
idx2country[27] = 'canada'
idx2country[23] = 'uniter_kingdom'
idx2country[3] = 'hungary'
idx2country[21] = 'switzerland'


In [0]:
  test_path = '/content/drive/My Drive/test'
  di = {}
  for img in os.listdir(test_path):
    idx = img.split('.')[0]
  
    img_path = os.path.join(test_path,img)
    IMG = cv2.imread(img_path)
    IMG = cv2.resize(IMG, (256,256))
    IMG = np.array(IMG).astype('float32') / 255.
    IMG = np.expand_dims(IMG, axis = 0)
    IMG = np.vstack([IMG])
    #pred = loaded_model.predict([IMG])
    #Id = np.where(pred.max)
    Class = model.predict_classes(IMG, batch_size=1)
    ID = (Class[0])+1
    #print(ID)
    #print(idx)
    #idx2country[Class[0]+1] = k
    #print("##########################")
    
    #if ID == '0':
      #print("Error")
    print (idx2country[ID])
    di[idx] = idx2country[ID]

pakistan
israel
uniter_kingdom
norway
thailand
chile
czech_republic
hungary
thailand
israel
indonesia
taiwan
taiwan
philippines
usa
switzerland
singapore
uniter_kingdom
chile
spain
south_africa
chile
poland
russia
spain
denmark
india
singapore
norway
spain
switzerland
denmark
chile
new_zealand
malaysia
poland
hungary
russia
denmark
new_zealand
turkey
israel
turkey
south_korea
taiwan
malaysia
thailand
hong_kong
australia
taiwan
new_zealand
mexico
australia
indonesia
south_africa
new_zealand
czech_republic
poland
uniter_kingdom
taiwan
turkey
poland
denmark
thailand
south_korea
mexico
india
uniter_kingdom
pakistan
thailand
new_zealand
uniter_kingdom
uniter_kingdom
canada
china
singapore
russia
denmark
spain
denmark
canada
denmark
spain
poland
australia
south_africa
south_africa
malaysia
mexico
denmark
singapore
hungary
russia
czech_republic
brazil
australia
uniter_kingdom
russia
chile
singapore
chile
malaysia
mexico
canada
philippines
czech_republic
israel
mexico
singapore
china
norway
tu

In [0]:
import csv
#sorted(di.keys())
os.chdir("/content/drive/My Drive")
with open('result_coin.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(('ID', 'Country'))
with open('result_coin.csv', 'a') as f:
    for key in di.keys():
      f.write("%s,%s\n"%(key,di[key]))

'hong_kong'

In [0]:
# idx2country = {}
# test_path = '/content/drive/My Drive/Colab Notebooks/coin_train_x'
# for idx in os.listdir(test_path):
#   idx_path = os.path.join(test_path,idx)
#   #print(os.listdir(idx_path)[0])
#   l = ((os.listdir(idx_path))[0]).split('.')[0]
#   p = l.split(' ')[-1]
#   q = p.split('_')
#   q = q[1:]
#   k = ""
#   if len(q) == 2:
#     k = q[0] + '_' + q[1]
#   else:
#     k = q[0]
#   print(k)
#   country = k
  
#   idx2country[idx] = country


australia
brazil
united_kingdom
canada
chile
china
czech_republic
denmark
spain
hong_kong
hungary
india
indonesia
israel
japan
south_korea
malaysia
mexico
new_zealand
norway
pakistan
philippines
poland
russia
singapore
south_africa
sweden
switzerland
taiwan
thailand
turkey
usa


In [0]:
idx2country['3']

'united_kingdom'